In [1]:
import torch
import dgl
from module import load_token_mapping
from dgl import save_graphs, load_graphs
import dgl


In [2]:
i2q, q2i = load_token_mapping('data/mt_new/queries')
i2s, s2i = load_token_mapping('data/mt_new/spus')

In [7]:
src, dst = [], []
with open('data/mt_new/new_train', 'r') as f:
    for line in f:
        q, s, label = line.strip().split('\t')
        label = int(label)
        if label == 1:
            src.append(q2i[q.strip()])
            dst.append(s2i[s.strip()])
# with open('data/mt_new/one_edit_graph', 'r') as f:
#     for line in f:
#         s, d = line.strip().split('\t')
#         src.append(q2i[s.strip()])
#         dst.append(q2i[d.strip()])
g = dgl.graph((src, dst), num_nodes=len(i2q))
bg = dgl.to_bidirected(g)
new_g = dgl.add_self_loop(bg)
save_graphs('data/mt_new/one_edit_q2q.graph', [new_g], None)

In [4]:
src

[2,
 2,
 3,
 5,
 6,
 6,
 6,
 6,
 6,
 7,
 7,
 7,
 8,
 8,
 8,
 8,
 9,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 11,
 12,
 15,
 17,
 17,
 18,
 18,
 18,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 22,
 22,
 22,
 22,
 23,
 23,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 27,
 27,
 29,
 29,
 29,
 30,
 31,
 31,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 3

In [55]:
graph_data = {
   ('query', 'q2s', 'spu'): (torch.tensor(src), torch.tensor(dst)),
   ('spu', 's2q', 'query'): (torch.tensor(dst), torch.tensor(src)),
}

In [56]:
g = dgl.heterograph(graph_data)

In [57]:
new_g = dgl.metapath_reachable_graph(g, ['q2s', 's2q'])

In [58]:
new_g = dgl.remove_self_loop(new_g)
new_g = dgl.add_self_loop(new_g)

In [59]:
new_g = dgl.to_homogeneous(new_g)

In [2]:
from dgl import save_graphs, load_graphs
import dgl

In [60]:
save_graphs('data/mt_new/train_q2q.graph', [new_g], None)

In [3]:
g = dgl.load_graphs('data/mt_new/train_q2q.graph')
new_g = g[0][0]
sg = dgl.sampling.sample_neighbors(new_g, [100, 342], 1)

In [4]:
sg, wm = dgl.to_simple(new_g, return_counts='cnt', writeback_mapping=True)

In [9]:
query = 9877
src = new_g.in_edges(query)[0].tolist()
print(i2q[query])
print([i2q[i] for i in src])

防风被夹
['防风夹', '防风被夹']


In [24]:
q = torch.arange(0, new_g.num_nodes())[new_g.in_degrees() > 1]

In [27]:
q[:100]

tensor([  1,   7,   9,  13,  16,  18,  22,  29,  35,  38,  46,  49,  52,  54,
         59,  60,  65,  66,  69,  73,  74,  89,  95,  99, 105, 106, 107, 108,
        109, 111, 118, 123, 130, 139, 140, 141, 145, 146, 148, 155, 159, 161,
        164, 165, 175, 179, 183, 184, 185, 188, 193, 199, 204, 224, 227, 235,
        246, 248, 250, 251, 257, 258, 261, 269, 270, 273, 274, 277, 281, 282,
        284, 289, 291, 299, 303, 305, 307, 320, 324, 327, 328, 335, 339, 342,
        343, 345, 346, 351, 357, 369, 370, 371, 373, 377, 378, 381, 383, 385,
        386, 388])

In [5]:
def build_pos_neg_graph():
    pos_src, pos_dst = [], []
    neg_src, neg_dst = [], []
    with open('data/mt_new/new_train', 'r') as f:
        for line in f:
            q, s, label = line.strip().split('\t')
            label = int(label)
            if label == 1:
                pos_src.append(q2i[q.strip()])
                pos_dst.append(s2i[s.strip()])
            else:
                neg_src.append(q2i[q.strip()])
                neg_dst.append(s2i[s.strip()])
    graph_data = {
        ('query', 'pos', 'spu'): (torch.tensor(pos_src), torch.tensor(pos_dst)),
        ('query', 'neg', 'spu'): (torch.tensor(neg_src), torch.tensor(neg_dst)),
    }
    g =  dgl.heterograph(graph_data, num_nodes_dict={'query': len(q2i), 'spu': len(s2i)})
    save_graphs('data/mt_new/pos_neg_train.graph', [g], None)
    return g

In [3]:
# pos_neg_g = build_pos_neg_graph()
pos_neg_g, _ = load_graphs('data/mt_new/pos_neg_train.graph')
pos_neg_g = pos_neg_g[0]

In [4]:
pos_neg_g

Graph(num_nodes={'query': 218758, 'spu': 1678075},
      num_edges={('query', 'neg', 'spu'): 785168, ('query', 'pos', 'spu'): 1151352},
      metagraph=[('query', 'spu', 'neg'), ('query', 'spu', 'pos')])

In [21]:
neg_g = dgl.to_homogeneous(pos_neg_g['neg'])
neg_g = dgl.to_bidirected(neg_g)
neg_g = dgl.add_self_loop(neg_g)

In [17]:
neg_g.edges()

(tensor([  72674,  171652,   99613,  ..., 1896830, 1896831, 1896832]),
 tensor([1697111, 1468970,  814484,  ..., 1896830, 1896831, 1896832]))

In [6]:
sg = dgl.sampling.sample_neighbors(neg_g, [0, 1], 1, edge_dir='out')

In [12]:
sg.edges(form='all')

(tensor([0, 1]), tensor([1619549,       1]), tensor([0, 1]))

In [18]:
tmp_g = dgl.remove_edges(sg, eids=[0])
tmp_g = dgl.add_edges(tmp_g, [72674], [1697111])

In [22]:
tmp_g.edges(form='all')

(tensor([    1, 72674]), tensor([      1, 1697111]), tensor([0, 1]))

In [20]:
tmp_g.edata[dgl.EID]

tensor([785169,      0])

In [16]:
tmp_g.edata[dgl.EID]

tensor([785169])

In [10]:
sg.edata

{'_ID': tensor([151210, 785169]), '_TYPE': tensor([0, 0])}

In [46]:
neg_src, neg_dst = sg.edges()
mask = neg_src == neg_dst
neg_dst[mask] = torch.randint_like(neg_dst[mask], low=218758, high=neg_g.num_nodes())

In [47]:
neg_dst

tensor([673688, 383617])

In [31]:
sg = dgl.sampling.sample_neighbors(pos_neg_g, {'query': [0, 1]}, {'pos': 1, 'neg': 0}, edge_dir='out')

In [33]:
sg['pos'].edges()[0].dtype

torch.int64

In [8]:
g = dgl.graph(([0], [0]), num_nodes=3)

In [12]:
sg = dgl.sampling.sample_neighbors(g, [0, 0, 2], 1)

In [13]:
sg.edges()

(tensor([0, 0]), tensor([0, 0]))